In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Importing the dataset
dataset = pd.read_csv('AIML Dataset.csv')
dataset.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(dataset['type'])
label

array([3, 3, 4, ..., 1, 4, 1])

In [5]:
# Inserting converted type data into our dataset
dataset.insert(2,'type_num', label)

In [6]:
# Converting nameOrig column into only ids by removing C from its front
dataset['nameOrig'] = dataset['nameOrig'].replace({'C': ''}, regex=True)
dataset

,step,type,type_num,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,3,9839.64,1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,3,1864.28,1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,4,181.00,1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,1,181.00,840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,3,11668.14,2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,1,339682.13,786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,4,6311409.28,1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,1,6311409.28,1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,4,850002.52,1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [7]:
dataset['nameOrig'] = pd.to_numeric(dataset['nameOrig'])

In [8]:
dataset = dataset.drop(['step', 'type','isFlaggedFraud'], axis=1)
dataset

,type_num,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,3,9839.64,1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0
1,3,1864.28,1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0
2,4,181.00,1305486145,181.00,0.00,C553264065,0.00,0.00,1
3,1,181.00,840083671,181.00,0.00,C38997010,21182.00,0.00,1
4,3,11668.14,2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...
6362615,1,339682.13,786484425,339682.13,0.00,C776919290,0.00,339682.13,1
6362616,4,6311409.28,1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1
6362617,1,6311409.28,1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1
6362618,4,850002.52,1685995037,850002.52,0.00,C2080388513,0.00,0.00,1


In [9]:
# we will be taking only these types into our data for analysis as they are the only one which have fraudulent transactions
fraud_cashout = dataset.loc[(dataset.isFraud == 1) & (dataset['type_num'] == 1)]
fraud_transfer = dataset.loc[(dataset.isFraud == 1) & (dataset['type_num'] == 4)]
print(len(fraud_cashout))
print(len(fraud_transfer))

4116
4097


In [11]:
# finding how many not fraudulent transactions cash out and transfers have
data1 = dataset.loc[(dataset.isFraud == 0) & (dataset.type_num ==1)]
data2 = dataset.loc[(dataset.isFraud == 0) & (dataset.type_num ==4)]

In [12]:
# #reshaping the data randomly to balance the dataset and reduce overfitting
# data1 = dataset.loc[(dataset.isFraud == 0) & (dataset['type'] == 'CASH_OUT')] #.sample(frac=0.002)
# data2 = dataset.loc[(dataset.isFraud == 0) & (dataset['type'] == 'TRANSFER')] #.sample(frac=0.008)

In [15]:
print(len(data1))
print(len(data2))

2233384
528812


In [14]:
# creating our data set by concatenating the segregated data frames
fraud_data1 = pd.concat([data1,fraud_cashout])
fraud_data1

,type_num,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
15,1,229133.94,905080434,15325.00,0.0,C476402209,5083.00,51513.44,0
42,1,110414.71,768216420,26845.41,0.0,C1509514333,288800.00,2415.16,0
47,1,56953.90,1570470538,1942.02,0.0,C824009085,70253.00,64106.18,0
48,1,5346.89,512549200,0.00,0.0,C248609774,652637.00,6453430.91,0
51,1,23261.30,2072313080,20411.53,0.0,C2001112025,25742.00,0.00,0
...,...,...,...,...,...,...,...,...,...
6362611,1,63416.99,994950684,63416.99,0.0,C1662241365,276433.18,339850.17,1
6362613,1,1258818.82,1436118706,1258818.82,0.0,C1240760502,503464.50,1762283.33,1
6362615,1,339682.13,786484425,339682.13,0.0,C776919290,0.00,339682.13,1
6362617,1,6311409.28,1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1


In [19]:
fraud_data1_1 = fraud_data1[:1000000]
fraud_data1_1

,type_num,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
15,1,229133.94,905080434,15325.00,0.0,C476402209,5083.00,51513.44,0
42,1,110414.71,768216420,26845.41,0.0,C1509514333,288800.00,2415.16,0
47,1,56953.90,1570470538,1942.02,0.0,C824009085,70253.00,64106.18,0
48,1,5346.89,512549200,0.00,0.0,C248609774,652637.00,6453430.91,0
51,1,23261.30,2072313080,20411.53,0.0,C2001112025,25742.00,0.00,0
...,...,...,...,...,...,...,...,...,...
2816181,1,272412.75,1525119195,0.00,0.0,C7354295,456391.73,728804.48,0
2816182,1,347890.73,1760267535,0.00,0.0,C1906897029,591084.46,938975.18,0
2816183,1,15639.78,1858709863,0.00,0.0,C1855175178,108884.29,124524.06,0
2816184,1,141504.16,1085670917,0.00,0.0,C37463778,191245.84,332750.00,0


In [20]:
fraud_data1_2 = fraud_data1[1000000:]
fraud_data1_2

,type_num,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
2816186,1,34518.82,356646316,0.00,0.0,C1925511667,851831.14,886349.96,0
2816187,1,278568.31,912325874,0.00,0.0,C1279988250,641896.48,920464.79,0
2816188,1,475369.51,1184256533,0.00,0.0,C231840997,1201817.38,1677186.90,0
2816189,1,43169.91,1401849968,0.00,0.0,C1993745946,896639.91,939809.83,0
2816190,1,251541.04,1236841264,0.00,0.0,C2117154478,3947280.39,4198821.43,0
...,...,...,...,...,...,...,...,...,...
6362611,1,63416.99,994950684,63416.99,0.0,C1662241365,276433.18,339850.17,1
6362613,1,1258818.82,1436118706,1258818.82,0.0,C1240760502,503464.50,1762283.33,1
6362615,1,339682.13,786484425,339682.13,0.0,C776919290,0.00,339682.13,1
6362617,1,6311409.28,1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1


In [21]:
fraud_data1_1.to_csv('fraud_data1_1_1.csv')
fraud_data1_2.to_csv('fraud_data1_1_2.csv')

In [23]:
fraud_data2 = pd.concat([data2, fraud_transfer])
fraud_data2

,type_num,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
19,4,215310.30,1670993182,705.00,0.0,C1100439041,22425.00,0.00,0
24,4,311685.89,1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0
58,4,62610.80,1976401987,79114.00,16503.2,C1937962514,517.00,8383.29,0
78,4,42712.39,283039401,10363.39,0.0,C1330106945,57901.66,24044.18,0
79,4,77957.68,207471778,0.00,0.0,C1761291320,94900.00,22233.65,0
...,...,...,...,...,...,...,...,...,...
6362610,4,63416.99,778071008,63416.99,0.0,C1812552860,0.00,0.00,1
6362612,4,1258818.82,1531301470,1258818.82,0.0,C1470998563,0.00,0.00,1
6362614,4,339682.13,2013999242,339682.13,0.0,C1850423904,0.00,0.00,1
6362616,4,6311409.28,1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1


In [24]:
fraud_data2.to_csv('fraud_data2.csv')